# Assignment 1.3: Naive word2vec (40 points)

This task can be formulated very simply. Follow this [paper](https://arxiv.org/pdf/1411.2738.pdf) and implement word2vec like a two-layer neural network with matrices $W$ and $W'$. One matrix projects words to low-dimensional 'hidden' space and the other - back to high-dimensional vocabulary space.

![word2vec](https://i.stack.imgur.com/6eVXZ.jpg)

You can use TensorFlow/PyTorch and code from your previous task.

## Results of this task: (30 points)
 * trained word vectors (mention somewhere, how long it took to train)
 * plotted loss (so we can see that it has converged)
 * function to map token to corresponding word vector
 * beautiful visualizations (PCE, T-SNE), you can use TensorBoard and play with your vectors in 3D (don't forget to add screenshots to the task)

## Extra questions: (10 points)
 * Intrinsic evaluation: you can find datasets [here](http://download.tensorflow.org/data/questions-words.txt)
 * Extrinsic evaluation: you can use [these](https://medium.com/@dataturks/rare-text-classification-open-datasets-9d340c8c508e)

Also, you can find any other datasets for quantitative evaluation.

Again. It is **highly recommended** to read this [paper](https://arxiv.org/pdf/1411.2738.pdf)

Example of visualization in tensorboard:
https://projector.tensorflow.org

Example of 2D visualisation:

![2dword2vec](https://www.tensorflow.org/images/tsne.png)

# Solution

## Imports and constants

In [1]:
import numpy as np
import torch
from torch import nn

from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import cosine as cosine_distance

from IPython.display import clear_output
import matplotlib.pyplot as plt

USE_COLAB = False
USE_CUDA = False
MOUNT_DIR = '/content/drive/'

In [2]:
if USE_COLAB:
    from google.colab import files, drive
    src = list(files.upload().values())[0]
    open('batcher.py','wb').write(src)
    drive.mount(MOUNT_DIR)
from batcher import SkipGramBatcherBase, read_corpus

In [3]:
if USE_CUDA and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print("device: ", device)

device:  cpu


## Skip-Gram Batcher impletation

In [4]:
class SkipGramBatcher(SkipGramBatcherBase):
    def __next__(self):
        """ Return next batch with order specified in self._permuted_indxs
            Return:
                centrals (np.array()): batch with tokens of central words. The size is (batch_size, )
                neighbours (np.array()): batch with tokens of neighbour words. 
                                         One word for each central word. The size is (batch_size, 1)
        """
        centrals, neighbours = self._get_next_batch()
        if centrals is None or neighbours is None:
            raise StopIteration
        
        rand_neighbours_idxs = np.random.randint(0, 2*self._window_size)
        rand_neighbours = neighbours[np.arange(self._batch_size), rand_neighbours_idxs]  # (batch_size, 1)
        
        return centrals, rand_neighbours

## Naive word2vec Pytorch module implementation

In [5]:
class NaiveWord2vec(nn.Module):
    def __init__(self, voc_size, embedding_dim):
        super(NaiveWord2vec, self).__init__()

        self.embedding_layer = nn.Embedding(voc_size, embedding_dim)
        self.linear_layer = nn.Linear(embedding_dim, voc_size, bias=False)
        self.activation = nn.LogSoftmax(dim=1)
    
    def forward(self, x):
        """ Forward pass of batch x
            Params:
                x (np.array): Batch with one-hot encoded vectors. The size is (batch_size, voc_size)
            Return:
                x (np.array): Batch with predicted neighbours. The size is (batch_size, voc_size)
        """
        x = self.embedding_layer(x)  # x: (batch_size, embedding_dim)
        x = self.linear_layer(x)  # x: (batch_size, voc_size)
        x = self.activation(x)  # x: (batch_size, voc_size)
        return x
    
    def get_embeddings_matrix(self):
        matrix = (
                    self.embedding_layer.cpu().weight.data.numpy() +
                    self.linear_layer.cpu().weight.data.numpy()
                 )
        return matrix

# Learning process

## Define params

In [6]:
WINDOW_SIZE = 5
BATCH_SIZE = 2048
VOCABULARY_SIZE = 30000
EMBEDDINGS_DIM = 200

START_EPOCH = 6
EPOCH_NUM = 6
DROW_EVERY_PERCENT = 1  # percent of total batches

TEXT_PATH = MOUNT_DIR + 'My Drive/datasets/nlp-ipavlov/text8' if USE_COLAB else 'data/text8'
MODELS_FOLDER = MOUNT_DIR + 'My Drive/models/nlp-ipavlov/' if USE_COLAB else 'models/'

## Preparation

### load data

In [7]:
corpus = read_corpus(TEXT_PATH)

In [8]:
dataset = SkipGramBatcher(corpus, WINDOW_SIZE, BATCH_SIZE, VOCABULARY_SIZE)

### create model

In [9]:
if START_EPOCH:
    model = torch.load(f'{MODELS_FOLDER}task3_word2vec_{START_EPOCH}_epoch.{VOCABULARY_SIZE}.{EMBEDDINGS_DIM}', map_location=torch.device('cpu'))
else:
    model = NaiveWord2vec(VOCABULARY_SIZE, EMBEDDINGS_DIM)
model.to(device)

NaiveWord2vec(
  (embedding_layer): Embedding(30000, 200)
  (linear_layer): Linear(in_features=200, out_features=30000, bias=False)
  (activation): LogSoftmax()
)

### create loss function

In [10]:
criterion = torch.nn.NLLLoss()

### create optimizer

In [11]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

## Learn process

In [12]:
%%time
sliding_loss = []
window_loss = []
for epoch in range(START_EPOCH, EPOCH_NUM):
    for i, (context, target) in enumerate(dataset):
        tensor_context = torch.from_numpy(context).type(torch.cuda.LongTensor if USE_CUDA else torch.LongTensor)
        tensor_target = torch.from_numpy(target).type(torch.cuda.LongTensor if USE_CUDA else torch.LongTensor)
        tensor_context.to(device)
        tensor_target.to(device)

        model.zero_grad()
        pred = model(tensor_context)
        loss = criterion(pred, tensor_target)
        loss.backward()
        optimizer.step()

        single_loss = loss.cpu().detach().numpy()
        window_loss.append(single_loss)
        if len(window_loss) > 2 * BATCH_SIZE:
            window_loss.pop(0)
        if i % int(1.0 * len(dataset) / 100 * DROW_EVERY_PERCENT) == 0:
            sliding_loss.append(np.average(window_loss))

            plt.figure(figsize=(13, 7))
            plt.xlabel('% of data')
            plt.ylabel('Loss')
            plt.title("Epoch number {}/{}".format(epoch+1, EPOCH_NUM))
            plt.plot(sliding_loss, label="average loss on last {} batches".format(2*BATCH_SIZE))
            clear_output(wait=True)
            plt.legend()
            plt.show()
    torch.save(model, f'{MODELS_FOLDER}task3_word2vec_{epoch+1}_epoch.{VOCABULARY_SIZE}.{EMBEDDINGS_DIM}')

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 7.87 µs
Parser   : 266 ms


# Evaluate model

In [16]:
class WordWeb(object):
    def __init__(self, embed_matrix, words2tokens, tokens2words, n_neighbours=5):
        self._matrix = embed_matrix
        self._words2tokens = words2tokens
        self._tokens2words = tokens2words
        self._neighbours_model = NearestNeighbors(n_neighbors=n_neighbours, metric=cosine_distance)
        self._neighbours_model.fit(embed_matrix)
        self._matrix2d = TruncatedSVD(2, n_iter=10).fit_transform(embed_matrix)
    
    def words_to_embeddings(self, words):
        tokens = self._words2tokens(words)
        return self._matrix[tokens, :]
    
    def most_similar(self, positive=[], negative=[], k_neighbors=5):
        avg = np.zeros((1, self._matrix.shape[1]))
        
        if len(positive):
            avg += np.mean(self.words_to_embeddings(positive), axis=0)
        if len(negative):
            avg -= np.mean(self.words_to_embeddings(negative), axis=0)

        similar = self._neighbours_model.kneighbors(avg, k_neighbors, return_distance=False)
        return self._tokens2words(similar[0])

In [17]:
wweb = WordWeb(model.get_embeddings_matrix(), dataset.words_to_tokens, dataset.tokens_to_words)

In [21]:
wweb.most_similar(positive=['france', 'berlin'], negative=['paris'])

['east', 'monarchies', 'expansion', 'berlin', 'republics']